# Шаблон архитектуры системы управления данными "Lakehouse"

## Мотивация

### Хранилища Данных - Data Warehouse, DWH

**Хранилища данных** (**Data Warehouse**) исторически были призваны решить задачу формирования аналитической картины для бизнеса на базе собираемой информации. Аналитика позволяет бинесу принимать взвешанные решения и формировать стратегию развития компании. Хранилища данных предъявляют требования к формату входящих данных, что позволяет пользователям хранилища данных работать с понятными данными. Характерной чертой традицонных хранилищ данных (**Data Warehouse**) является неразрывная связь между вычислительными ресурсами и ресурсами для хранения данных.

С увеличением объема поступающей информации традицонный подход к формированию хранилища данных столкнулись с большими проблемами: неразрывность слоёв хранения и обработки данных диктовала бизнесу вертикальное масштабирование. Бизнес был вынужен закупать железо, способное обслуживать пиковые нагрузки. Таким образом, бизнес тратил много денег, хотя пиковые нагрузки могут и не возникать часто.

В дополнение к увеличению объема обрабатываемой информации альтернативные форматы данных, которые не всегда имеют чёткую структуру (видео, музыка, текст и т.д.), также начали играть большую роль в построении аналитической картины. К неструктурированным данным **хранилища данных** были не готовы абсолютно.

### Озёра Данных - Data Lake

Для решения накопившихся проблем были разработаны платформы аналитики данных **второго поколения**.

Основная идея заключается в том, что все поступающие данные начали складываться в **озёра данных** (**Data Lake**) - кластер из дешёвых жестких дисков. Данные в **озёре данных** хранятся открытых (open source) форматах, таких как Apache Parquet или ORC.

Apache Hadoop с HDFS популяризровал подход с озером данных. От требования к формату входящих данных (`schema-on-write`) удалось перейти к `schema-on-read` - только конечный потребитель знает какие данные ему нужны, и он сам является ответственным за извлечение этих данных из озера данных. Таким образом, `ETL` (**Extract Transform Load**) был заменен на `ELT` (**Extract Load Transform**): бизнес смог собирать всю доступную информацию, которая потом анализировалась различными подходами.

Архитектура **Озеро Данных + Хранилища Данных** является доминирующей в современном мире.

Отличительной особенностью озера данных является разделение слоёв хранения данных и обработки данных: бизнес может заказывать дополнительные процессорные мощности при необходимости.

### Ограничения современной архитектуры

В свою очередь озера данных (Data Lake) принесли свои проблемы связанные с качеством и управлением данных. При этом данные по прежнему необходимо доставлять в хранилища данных, к которым подключены витрины данных. Таким образом, все поступающие данные проходят два или три шага:

- E**L**T для попадания в озеро данных (Data Lake),
- ET**L** для попадания в хранилище данных (Warehouse),
- (опиционально) ETL для машинного обучения.

Современний бизнес также полагается на системы машинного обучения и искуственного интеллекта, для которых ни озёра данных (Data Lake), ни хранилища данных (Data Warehouse) не являются идеальным решением.

Среди проблем современной архитектуры организации данных (**Озеро Данных** + **Хранилища Данных**) можно выделить следующие категории:

- **Надежность**,
- **Свежесть данных**,
- **Ограниченные возможности расширенной аналитики**,
- **Итоговая стоимость владения данными**.

#### Надежность

Задача поддежрки озера данных и хранилища данных в консистентном состоянии является сложной и затратной. Перекачка данных из озера данных в хранилище данных несёт риски связанные со сбоем оборудования или ошибками программиста, что может выражаться в некоторой разнице между озером данных и хранилищем данных, а это ухудшает качество данных.

#### Свежесть данных

Самые новые данные всегда сначала в озеро данных, а уже потом в хранилище. Перекачка данных из озера данных в хранилище не может выполниться мнгновенно. Таким образом, современные аналитические системы уступают своим предшественникам (**Data Warehouse**) по качеству свежести данных.

#### Ограниченные возможности расширенной аналитики

Бизнес должен иметь возможность задавать вопросы относительно будущего на основании собранных данных (например, "Кому эффективнее всего предоставить скидки?").

Современные фреймворки организации машинного обучения PyTorch, TensorFlow, XGBoost имеют ограниченные возможности работы на базе хранилища данных. В отличии от привычных хранилищам запросов BI характера, которые работают с небольшим объемом информации, фреймворки машинного обучения требуют обработки огромных объемов данных.

Для подключения машинного обучения есть два варианта:

- выгрузить данные из хранилища данных в файлы, которые можно использовать при работе с машинным обучением (третий ETL шаг),
- работа с озером данных напрямую. Это отключает возможности хранилища данных по управлению данными:
    - ACID транзакции,
    - версионирование данных,
    - индексирование.

#### Итоговая стоимость владения данными

Одновременная поддержка озера данных и хранилища данных фактически увеличивает затраты на хранение данных вдвое: одни и те же данные лежат как в озере данных ("грязные"), так и в хранилище данных ("чистые"). При этом также в два раза увеличиваются риски утечки данных.

### Требования к организации системы управления данными

Таким образом, необходимо организовать данные в озере данных, которое отвечает следующим требованиям:

1. открытость форматов данных (Parquet, ORC) для хранения как защита от Vendor Lock-In,
2. производительность сопоставимая с хранилищами данных,
3. возможности управления атрибутами/признаками (feature) данных, сопоставимые с хранилищами данных,
4. прямой `I/O` доступ к данным для расширенной аналитики.

Шаблон архитектуры **Lakehouse** позволяет построить озеро данных, которое отвечает поставленным требованиям.

## Шаблон Lakehouse

Lakehouse предлагает организовать систему управления данными на базе следующих концепций:

- хранение данных в кластера из дешевых жестких дисков (HDFS, S3, GCS),
- система демонстрирует функции аналитической СУБД:
    - ACID транзакции,
    - версионирование данных,
    - аудит,
    - индексы,
    - кеширование,
    - оптимизатор запросов.

Таким образом, система построеная по шаблону Lakehouse заключает в себе комбинацию ключевых преимуществ озёр данных (Data Lake) и хранилищ данных (Data Warehouse):

- низкая стоимость хранения данных,
- открытые форматы данных (как в Data Lake),
- мощные возможности управления данными и оптимизации запросов (как в Data Warehouse).

### Концепции Lakehouse

Шаблон Lakehouse состоит из следующих концепций:

- все данные хранятся в блочном хранилище вроде HDFS или S3;
- все данные приводятся к стандартному формату, например, Parquet;
- данные могут быть организованы в виде логических таблиц;
- поверх слоя хранения данных реализован слой метаданных;
- слой метаданных может содержать:
    - статистику по файлам,
    - список файлов, относящихся к текущей версии таблицы.
- ACID транзакции, версионирование, индексация - манипуляции со слоем метаданных;
- клиенты могут читать файлы из HDFS или S3 напрямую.

### Реализация Lakehouse

Форматы [Delta Lake](https://delta.io/) (Databricks), [Apache Iceberg](https://iceberg.apache.org/) (Netflix) и [Apache Hudi](https://hudi.apache.org/) (Uber) реализуют концепции похожие на Lakehouse схожим образом:

- данные сохраняются в формате Parquet или ORC в блочное хранилище (HDFS, S3),
- сохраненные файлы неизменны,
- рядом с сохраненными данными создаеются файлы с метаданными,
- при внесении изменений в файлы, создаются новые файлы,
- метаданные отслеживают какие из имеющихся файлов относятся к текущей версии.

Формат Delta Lake был разработан в компании Databricks, которая развивает Apache Spark, поэтому построение системы управления данными на примере Lakehouse в практической части будет основываться на Delta Lake.

> **Delta Lake - основной формат для построения Lakehouse!**

### Ограничения Delta Lake

Один файл с метаданными относится к одному Parquet файлу с данными, поэтому, к сожалению, невозможно получить ACID транзакции, включающие множество файлов. Эта проблема известна и находится на контроле ответственных организаций.

Учитывая, что Databrics запускает примерно 50% (на 2021 год) всей нагрузки своих клиентов через Delta Lake, возможно отсутствие ACID транзакций на несколько файлов не является большой проблемой.

### Особенности Delta Lake

Вне зависимости от используемого блочного хранилища Delta Lake эффективно реализует оптимизации:

- пропуск файлов при сканировании,
- кластеризация строк.

#### Анализ статистики для пропуска файлов при сканировании

Вне зависимости от используемого блочного хранилища, файл с метаданными хранит информацию, которая может использоваться оптимизатором запросов. В метаданных хранится статистика о максимальных и минимальных значениях колонок для каждого файла, что позволяет исключать файлы из сканирования.

#### Пропуска файлов при сканировании на базе Bloom фильтра

Для каждого parquet файла строится Bloom фильтр, который также позволяет эффективно пропускать файлы, в которых точно нет требуемых данных.

#### Кластеризация строк

Delta Lake использует продвинутые алгоритмы для определения наилучшей организации строки на диске, с учетом, что несколько строк будут наиболее часто считываться вместе.

# Запуск Spark

## Конфигурация

Для работы с Delta Lake из Python необходимо установить пакет `delta-lake`:

In [3]:
! pip install delta-spark==3.0.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 2.3 MB/s eta 0:00:00a 0:00:01


### Запуск Apache Spark

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.functions import col
from pyspark.sql import functions as F

In [5]:
from delta import configure_spark_with_delta_pip
from delta.tables import DeltaTable

In [ ]:
builder = (
    SparkSession
        .builder
        .appName("Delta Demo")
        .master("local[4]")
        .config("spark.sql.warehouse.dir", "data/spark-warehouse")
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
)

Функция `configure_spark_with_delta_pip` активиурет возможности Delta Lake:

In [ ]:
print(configure_spark_with_delta_pip.__doc__)

In [ ]:
spark = configure_spark_with_delta_pip(builder) \
    .getOrCreate()

sc = spark.sparkContext

### Подготовка данных

Распаковать подготовленные архивы с данными:

In [ ]:
! cd /tmp && rm -rf steam taxi.parquet && unzip -o ~/work/data/steam.zip && unzip -o ~/work/data/taxi.zip

Функция `dump_parquet` скроет подробности созхранения датафрейма на диск как parquet:

In [ ]:
def dump_parquet(df: DataFrame, num: int, loc: str):
    (df
        .repartition(num)
        .write
        .mode("overwrite")
        .parquet(loc)
    )

Файлы будут располагаться в HDFS, поэтому необходимо очистить файловую систему:

In [ ]:
! source ~/.bash_aliases && \
hdfs dfs -rm -f -r /user/jovyan/data && \
hdfs dfs -mkdir -p /user/jovyan/data/stream && \
hdfs dfs -mkdir -p /user/jovyan/data/taxi

Загрузить локальные файлы в Apache Spark, разбить их на 4 партиции и положить в HDFS:

In [ ]:
sc.setJobDescription("Разбить датафреймы steam на 4 партиции")

# dump_parquet(spark.read.parquet("file:///tmp/steam/details.parquet"), 4, "/user/jovyan/data/steam/details")
# dump_parquet(spark.read.parquet("file:///tmp/steam/tags.parquet"), 4, "/user/jovyan/data/steam/tags")
dump_parquet(spark.read.parquet("file:///tmp/steam/games.parquet"), 4, "/user/jovyan/data/steam/games")
dump_parquet(spark.read.parquet("file:///tmp/taxi.parquet"), 4, "/user/jovyan/data/taxi")

Файлы загружены в HDFS:

In [ ]:
! source ~/.bash_aliases && \
hdfs dfs -find /user/jovyan/data

## Delta таблицы

In [ ]:
my_first_table_path = 'data/my-first-delta-table'

### Создание Delta таблицы

Сохранение датафрейма в формате `delta` создает новую Delta таблицу:

In [ ]:
data = spark.range(0, 5)
(
    data
        .write
        .format("delta")
        .save(my_first_table_path)
)

### Загрузка Delta таблицы

Чтение данных в формате `delta` читает данные из таблицы:

In [ ]:
df = (
    spark
        .read
        .format("delta")
        .load(my_first_table_path)
)
df.show()

### Обновление Delta таблицы

Запись датафрейма в формате `delta` и в режиме `overwrite` обновляет данные в `delta` таблице:

In [ ]:
df = spark.range(5, 10)
(
    df
        .write
        .format("delta")
        .mode("overwrite")
        .save(my_first_table_path)
)

In [ ]:
df = (
    spark
        .read
        .format("delta")
        .load(my_first_table_path)
)
df.show()

### Частичное обновление Delta таблицы

Данные в таблице можно обновить по условию:

In [ ]:
deltaTable = DeltaTable.forPath(spark, my_first_table_path)

deltaTable.update(
    condition = F.expr("id % 2 == 0"),
    set = { "id": F.expr("id + 100") }
)

In [ ]:
df = (
    spark
        .read
        .format("delta")
        .load(my_first_table_path)
)
df.show()

### Удаление данных Delta таблицы

Удалить данные в таблице можно можно обновить по условию:

In [ ]:
deltaTable = DeltaTable.forPath(spark, my_first_table_path)
deltaTable.delete(condition = F.expr("id % 2 == 0"))

In [ ]:
df = (
    spark
        .read
        .format("delta")
        .load(my_first_table_path)
)
df.show()

### Слияние данных - Merge, Upsert

Данные могут быть записаны в таблицу по принципу слияния (merge, upsert), т.е. существующие данные заменяются, новые данные добавляются:

In [ ]:
deltaTable = DeltaTable.forPath(spark, my_first_table_path)

# New data to upsert (merge) 
newData = spark.range(0, 20).withColumnRenamed("id", "x")

deltaTable \
  .merge(newData, col("id") == col("x")) \
  .whenMatchedUpdate(set = { "id": col("x") }) \
  .whenNotMatchedInsert(values = { "id": col("x") }) \
  .execute()

In [ ]:
df = (
    spark
        .read
        .format("delta")
        .load(my_first_table_path)
)
df.show(5)

Обратите внимание, что операция `execute` выполняется немедленно.

## Машина времени

Все файлы, загруженные в HDFS через Delta Lake, становятся неизменяемыми. Но как тогда выполняется обновление данных?

Delta создает новые файлы в директории таблицы и при помощи дополнительных файлов отслеживает, какие файлы составляют содержимое текущей версии таблицы.

Delta поддерживает версионирование, а значит в любой момент можно вернуться к любой из предыдущих версий: файлы по прежнему находятся в директории таблицы. В качестве примера можно вернуться к самой первой версии:

In [ ]:
df = (
    spark.read
        .format("delta")
        .option("versionAsOf", 0)
        .load(my_first_table_path)
)

df.show()

Также можно вернуться к предыдущей версии:

In [ ]:
deltaTable = DeltaTable.forPath(spark, my_first_table_path)
deltaTable.history().show()

In [ ]:
deltaTable.restoreToVersion(0)

In [ ]:
deltaTable.history().show()

In [ ]:
deltaTable.toDF().show()

## Конвертация Parquet в Delta Lake

Delta Lake работает на базе Parquet, поэтому конвертация выполняется простым добавлением файлов с метаданными в директорию с Parquet файлами:

Состояние директории без файлов с метаданными Delta Lake:

In [ ]:
! source ~/.bash_aliases && \
hdfs dfs -find /user/jovyan/data/taxi

In [ ]:
taxi_path = '/user/jovyan/data/taxi'

In [ ]:
deltaTable = DeltaTable.convertToDelta(spark, f"parquet.`{taxi_path}`")

In [ ]:
print(DeltaTable.convertToDelta.__doc__)

Состояние директории после конвертации в Delta Lake:

In [ ]:
! source ~/.bash_aliases && \
hdfs dfs -find /user/jovyan/data/taxi

Директория `_delta_log` содержит метаданные, в частности, схему:

In [ ]:
! source ~/.bash_aliases && \
hdfs dfs -cat /user/jovyan/data/taxi/_delta_log/00000000000000000000.json | \
grep metaData | json_pp | \
grep schema | sed 's,\s*"schemaString" : ",,;s,"$,,;s,\\",",g' | json_pp > schema.json

Вывод команды сохранен в [schema.json](schema.json)

In [ ]:
! source ~/.bash_aliases && \
hdfs dfs -cat /user/jovyan/data/taxi/_delta_log/00000000000000000000.json | \
grep add |& head -n 1 | json_pp | \
grep stats | sed 's,\s*"stats" : ",,;s,"$,,;s,\\",",g' | json_pp > stats.json

Вывод команды сохранен в [stats.json](stats.json)

## Z-Order оптимизация

In [ ]:
deltaTable = DeltaTable.forPath(spark, taxi_path)

In [ ]:
! source ~/.bash_aliases && \
hdfs dfs -find /user/jovyan/data/taxi

Оптимизация Z-Order позволяет сохранить в одном файле записи, которые будут часто выбираться вместе. Так, например, если часто нужно получать агрегированные значения среди поездок в такси с разным числом пассажиров, то имеет смысл выполнить Z-Order оптимизацию по колонке `passenger_count`:

In [ ]:
df = deltaTable.optimize().executeZOrderBy("passenger_count")

In [ ]:
! source ~/.bash_aliases && \
hdfs dfs -find /user/jovyan/data/taxi

In [ ]:
df.printSchema()

In [ ]:
df.select("metrics.zOrderStats").show(truncate=False, vertical=True)

In [ ]:
(
    df.select("metrics.zOrderStats.inputOtherFiles", "metrics.zOrderStats.mergedFiles")
        .select(
            col("inputOtherFiles.num").alias("input_num_files"), col("inputOtherFiles.size").alias("input_size"),
            col("mergedFiles.num").alias("merged_num_files"), col("mergedFiles.size").alias("merged_size")
        )
        .show(truncate=False)
)

In [ ]:
deltaTable.toDF().groupBy("passenger_count").count().explain()

In [ ]:
(
deltaTable.history()
    .select(
        "version",
        "timestamp",
        "operation",
        "operationMetrics.numRemovedFiles",
        "operationMetrics.numAddedFiles",
        "operationMetrics.numConvertedFiles")
    .show(truncate=False)
)

## Вакуумная очистка

Количество файлов таблицы может расти бесконтрольно, но не все из них нужны. Файлы, которые относятся к старым версиям можно удалять при помощи [`vacuum`](https://docs.delta.io/latest/api/python/index.html#delta.tables.DeltaTable.vacuum):

In [ ]:
deltaTable = DeltaTable.forPath(spark, taxi_path)

In [ ]:
import sys

try:
    deltaTable.vacuum(0)
except Exception as e:
    print(e, file=sys.stderr)

In [ ]:
spark.conf.set('spark.databricks.delta.retentionDurationCheck.enabled', False)

In [ ]:
deltaTable.vacuum(0)

In [ ]:
spark.conf.set('spark.databricks.delta.retentionDurationCheck.enabled', True)

In [ ]:
! source ~/.bash_aliases && \
hdfs dfs -find /user/jovyan/data/taxi

In [ ]:
deltaTable.history().show()

In [ ]:
import sys

try:
    (
    spark.read
        .format("delta")
        .option("versionAsOf", 0)
        .load(taxi_path)
        .show()
    )
except Exception as e:
    print(e.java_exception.getMessage(), file=sys.stderr)

## Построение Lakehouse архитектуры

In [ ]:
spark.stop()

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.functions import col
from pyspark.sql import functions as F

In [2]:
from delta import configure_spark_with_delta_pip
from delta.tables import DeltaTable

In [3]:
src_games_path = "data/steam/games"

base_path = "data/games"
source_path = base_path + "/source"

checkpoint_path = base_path + "/checkpoints"

In [4]:
! source ~/.bash_aliases && \
hdfs dfs -rm -r -f data/games

In [4]:
builder = (
    SparkSession
        .builder
        .appName("Delta Demo")
        .master("local[4]")
        .config("spark.sql.warehouse.dir", "data/spark-warehouse")
        .config("hive.metastore.uris", "thrift://hive:9083")
        .config("spark.driver.memory", "4g")
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
        .enableHiveSupport()
)

spark = configure_spark_with_delta_pip(builder) \
    .getOrCreate()

sc = spark.sparkContext

Для небольших данных имеет смысл установить значение числа партиций после Shuffle равным числу доступных процессоров:

In [5]:
spark.conf.set("spark.sql.shuffle.partitions", sc.defaultParallelism)

In [6]:
spark.catalog.listTables()

[Table(name='bronze_reviews_igromania', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='bronze_reviews_metacritic', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='games', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='golden_review_facts', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='silver_reviews', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='silver_reviews_igromania', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='silver_reviews_igromania_v', catalog='spark_catalog', namespace=['default'], description=None, tableType='VIEW', isTemporary=False),
 Tabl

In [7]:
games_delta = DeltaTable.convertToDelta(spark, f"parquet.`{src_games_path}`")
# games_delta.toDF().write.format("delta").mode("overwrite").saveAsTable("games")

In [8]:
from random import randrange

def generate_reviews(ids_range, rating, date, max_users=100000):
    ids = [ F.lit(x) for x in ids_range ]
    
    return (games_delta.toDF()
        .withColumn("id", F.explode(F.array(ids)))
        .select(
            F.expr("app_id * 1000000 + id").alias("id"),
            "app_id",
            date,
            col("title").alias("review"),
            rating,
            F.floor(F.rand() * max_users).alias("user_id"),
        )
    )

def generate_reviews_metacritic(ids_range):
    rating=F.floor(F.rand() * 100).alias("score")
    date=F.expr("STRING(DATE_ADD(date_release, CAST(floor(rand() * 500) AS INT)))").alias("review_date")
    return generate_reviews(ids_range, rating, date)

def generate_reviews_igromania(ids_range):
    rating=F.round(F.rand() * 10, 1).alias("rating")
    date=F.expr("unix_timestamp() + floor(rand() * 2 * 12 * 30 * 24 * 60 * 60)").alias("review_date_unix")
    return generate_reviews(ids_range, rating, date)

In [ ]:
reviews_metacritic = generate_reviews_metacritic(ids_range=range(1, 10))
reviews_igromania = generate_reviews_igromania(ids_range=range(1, 5))

In [ ]:
reviews_metacritic.show(5, truncate=False)
reviews_igromania.show(5, truncate=False)

In [9]:
def save_df(df, location, format="delta", mode="append"):
    (
    df
        .write
        .format(format)
        .mode(mode)
        .save(location)
    )

In [ ]:
save_df(reviews_metacritic, f"{source_path}/reviews_metacritic", format="json", mode='overwrite')
save_df(reviews_igromania, f"{source_path}/reviews_igromania", format="json", mode='overwrite')

Проверить, что нет дублей по ключу:

In [ ]:
def check_duplicates(location: str, format: str="delta"):
    (
    spark.read
        .format(format)
        .option("path", location)
        .load()
        .groupBy("id")
        .count()
        .where("count > 1")
        .show()
    )

In [ ]:
check_duplicates(f"{source_path}/reviews_metacritic", "json")

In [ ]:
check_duplicates(f"{source_path}/reviews_igromania", "json")

Сгенерировать пользователей:

In [ ]:
users_df = (
    spark.range(0, 100000)
        .withColumn("username", F.concat(F.lit("user"), F.lit("_"), col("id")))
        .withColumn("gender", F.when(F.rand() > 0.6, "F").otherwise("M"))
        .withColumn("dob", F.expr("ADD_MONTHS(current_date(), -CAST(floor(rand() * 25 * 12) AS INT) -10 * 12)"))
)
users_df.show(5)

In [ ]:
users_df.write.format("delta").mode("overwrite").saveAsTable("users")

### Bronze

In [ ]:
spark.sql("DROP TABLE IF EXISTS bronze_reviews_igromania")

In [ ]:
spark.sql("DROP TABLE IF EXISTS bronze_reviews_metacritic")

In [10]:
def ingest_raw_reviews(source_path: str, checkpoint_dir: str, sink_table_name: str):
    raw_reviews_read_stream = (
        spark.readStream
                .format("text")
                .schema("data STRING")
                .option("maxFilesPerTrigger", 1)
                .option("checkpointLocation", checkpoint_dir)
                .load(source_path)
                .withColumn("seen_time", F.current_timestamp())
    )
    
    return (
        raw_reviews_read_stream
            .writeStream
            .queryName(f"Lakehouse: {source_path} -> {sink_table_name}")
            .format("delta")
            .option("checkpointLocation", checkpoint_dir)
            .outputMode("append")
            .trigger(processingTime='5 seconds')
            .toTable(sink_table_name)
    )

In [11]:
bronze_reviews_igromania_stream = ingest_raw_reviews(
    source_path=f"{source_path}/reviews_igromania",
    checkpoint_dir=f"{checkpoint_path}/igromania",
    sink_table_name="bronze_reviews_igromania"
)

In [12]:
bronze_reviews_metacritic_stream = ingest_raw_reviews(
    source_path=f"{source_path}/reviews_metacritic",
    checkpoint_dir=f"{checkpoint_path}/metacritic",
    sink_table_name="bronze_reviews_metacritic"
)

In [ ]:
spark.table("bronze_reviews_igromania").show(5, vertical=True, truncate=False)

In [ ]:
spark.table("bronze_reviews_metacritic").show(5, vertical=True, truncate=False)

In [ ]:
spark.table("bronze_reviews_igromania").count()

In [ ]:
spark.table("bronze_reviews_metacritic").count()

In [ ]:
spark.table("bronze_reviews_igromania").count()

In [ ]:
spark.table("bronze_reviews_metacritic").count()

In [ ]:
spark.table("bronze_reviews_igromania").select(F.count_distinct("seen_time")).show()

### Silver

In [ ]:
spark.sql("DROP TABLE IF EXiSTS silver_reviews")

In [13]:
def ingest_bronze_reviews(bronze_table: str, checkpoint_dir: str, sink_table_name: str, json_parser, *cols):
    bronze_reviews_read_stream = (
        spark.readStream
                .format("delta")
                .option("maxFilesPerTrigger", 1)
                .option("checkpointLocation", checkpoint_dir)
                .table(bronze_table)
                .select("seen_time", json_parser)
                .select(*cols)
    )
    
    return (
        bronze_reviews_read_stream
            .writeStream
            .queryName(f"Lakehouse: {bronze_table} -> {sink_table_name}")
            .format("delta")
            .option("checkpointLocation", checkpoint_dir)
            .outputMode("append")
            .trigger(processingTime='5 seconds')
            .toTable(sink_table_name)
    )

In [ ]:
reviews_igromania.printSchema()

In [14]:
igromania_json_parser = F.from_json(
    "data",
    """
        id INT,
        app_id INT,
        review_date_unix LONG,
        review STRING,
        rating DOUBLE,
        user_id LONG
    """
).alias("payload")

silver_reviews_from_igromania_stream = ingest_bronze_reviews(
    "bronze_reviews_igromania",            # `bronze_table` param
    f"{checkpoint_path}/igromania_bronze", # `checkpoint_dir` param
    "silver_reviews",                      # `sink_table_name` param
    igromania_json_parser,                 # `json_parser` param

    # columns to select after parsing the json:
    "seen_time",
    F.concat(F.lit("IGROMANIA"), col("payload.id")).alias("id"),
    "payload.app_id",
    F.from_unixtime("payload.review_date_unix").cast("TIMESTAMP").alias("review_ts"),
    "payload.review",
    F.floor(col("payload.rating") * 10).alias("rating"),
    "payload.user_id",
    F.lit("IGROMANIA").alias("source")
)

In [19]:
spark.table("silver_reviews").count()

368544

In [ ]:
reviews_metacritic.printSchema()

In [15]:
metacritic_json_parser = F.from_json(
    "data",
    """
        id LONG,
        app_id INT,
        review_date STRING,
        review STRING,
        score INT,
        user_id LONG
    """
).alias("payload")

silver_reviews_from_metacritic_stream = ingest_bronze_reviews(
    "bronze_reviews_metacritic",            # `bronze_table` param
    f"{checkpoint_path}/metacritic_bronze", # `checkpoint_dir` param
    "silver_reviews",                       # `sink_table_name` param
    metacritic_json_parser,                 # `json_parser` param

    # columns to select after parsing the json:
    "seen_time",
    F.concat(F.lit("METACRITIC"), col("payload.id")).alias("id"),
    "payload.app_id",
    F.to_timestamp("payload.review_date").alias("review_ts"),
    "payload.review",
    F.floor(col("payload.score")).alias("rating"),
    "payload.user_id",
    F.lit("METACRITIC").alias("source")
)

In [36]:
spark.table("silver_reviews").count()

368544

In [ ]:
spark.table("silver_reviews").show(10, truncate=False)

### Gold

In [ ]:
spark.sql("DROP TABLE IF EXISTS golden_review_facts")

In [16]:
def ingest_silver_reviews(silver_table: str, checkpoint_dir: str, sink_table_name: str, *cols):
    silver_reviews_read_stream = (
        spark.readStream
                .format("delta")
                .option("maxFilesPerTrigger", 1)
                .option("checkpointLocation", checkpoint_dir)
                .table(silver_table)
                .select(*cols)
    )
    
    return (
        silver_reviews_read_stream
            .writeStream
            .queryName(f"Lakehouse: {silver_table} -> {sink_table_name}")
            .format("delta")
            .option("checkpointLocation", checkpoint_dir)
            .outputMode("append")
            .trigger(processingTime='5 seconds')
            .toTable(sink_table_name)
    )

In [ ]:
spark.table("silver_reviews").printSchema()

In [ ]:
spark.table("silver_reviews").select(col("id").alias("review_id"), "app_id", "user_id").show(5, truncate=False)

In [17]:
golden_review_facts_stream = ingest_silver_reviews(
    "silver_reviews",                         # `silver_table` param
    f"{checkpoint_path}/review_facts_golden", # `checkpoint_dir` param
    "golden_review_facts",                    # `sink_table_name` param

    # columns for dimensions
    col("id").alias("review_id"),
    "app_id",
    "user_id"
)

In [34]:
spark.table("golden_review_facts").count()

368544

In [ ]:
spark.catalog.listTables()

In [ ]:
review_facts = spark.table("golden_review_facts")
users = spark.table("users")

In [ ]:
users.where("id % 1000 = 2").show(5)

In [ ]:
reviews_users = (
    review_facts.alias("f")
        .join(
            users.alias("u"),
            col("f.user_id") == col("u.id")
        )
        .where("u.id % 1000 = 1")
)
reviews_users.count()

In [ ]:
reviews_users.show(5, truncate=False)

In [18]:
reviews_metacritic = generate_reviews_metacritic(ids_range=range(30, 40))
reviews_igromania = generate_reviews_igromania(ids_range=range(21, 25))

In [19]:
save_df(reviews_metacritic, f"{source_path}/reviews_metacritic", format="json", mode='append')
save_df(reviews_igromania, f"{source_path}/reviews_igromania", format="json", mode='append')

In [ ]:
reviews_users.count()

In [ ]:
spark.sql("select * from golden_review_facts").show()

### Настроить Apache Superset

In [47]:
! source ~/.bash_aliases && HOST=superset execute \
pip install trino && docker compose restart superset

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
 Container superset  Restarting
 Container superset  Started


In [37]:
! source ~/.bash_aliases && HOST=superset execute \
superset fab create-admin \
              --username admin \
              --password admin \
              --firstname Superset \
              --lastname Admin \
              --email 'admin@superset.com'

Debug mode identified with default secret key
--------------------------------------------------------------------------------
                                    WARNING
--------------------------------------------------------------------------------
A Default SECRET_KEY was detected, please use superset_config.py to override it.
Use a strong complex alphanumeric string and use a tool to help you generate 
a sufficiently random sequence, ex: openssl rand -base64 42
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
2023-10-22 05:28:16,988:INFO:superset.utils.logging_configurator:logging was configured successfully
2023-10-22 05:28:16,997:INFO:root:Configured event logger of type <class 'superset.utils.log.DBEventLogger'>
/usr/local/lib/python3.9/site-packages/flask_limiter/extension.py:293: UserWarning: Using the in-memory storage for tracking rate limits as no storage was ex

In [ ]:
! source ~/.bash_aliases && HOST=superset execute \
superset db upgrade

In [39]:
! source ~/.bash_aliases && HOST=superset execute \
superset init

Debug mode identified with default secret key
--------------------------------------------------------------------------------
                                    WARNING
--------------------------------------------------------------------------------
A Default SECRET_KEY was detected, please use superset_config.py to override it.
Use a strong complex alphanumeric string and use a tool to help you generate 
a sufficiently random sequence, ex: openssl rand -base64 42
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
2023-10-22 05:28:49,975:INFO:superset.utils.logging_configurator:logging was configured successfully
2023-10-22 05:28:49,988:INFO:root:Configured event logger of type <class 'superset.utils.log.DBEventLogger'>
/usr/local/lib/python3.9/site-packages/flask_limiter/extension.py:293: UserWarning: Using the in-memory storage for tracking rate limits as no storage was ex

In [53]:
import requests
import json

In [54]:
payload = {
  "username": "admin",
  "password": "admin",
  "provider": "db"
}
r = requests.post('http://superset:8088/api/v1/security/login', json=payload)

assert r.status_code == 200

access_token = r.json()["access_token"]

In [55]:
headers = {'Authorization': f'Bearer {access_token}'}
r = requests.get('http://superset:8088/api/v1/security/csrf_token/', headers=headers)

assert r.status_code == 200

csrf = r.json()["result"]
cookies = r.cookies

In [56]:
headers = {
    "Authorization": f"Bearer {access_token}",
    "X-CSRFToken": csrf
}

In [57]:
payload = {
    "database_name": "Trino Lakehouse",
    "sqlalchemy_uri": "trino://trino@trino:8080/delta_lake",
    "engine": "trino"
}

r = requests.post('http://superset:8088/api/v1/database', headers=headers, json=payload, cookies=cookies)

assert r.status_code == 201

r.json()

{'id': 2,
 'result': {'configuration_method': 'sqlalchemy_form',
  'database_name': 'Trino Lakehouse',
  'driver': 'rest',
  'expose_in_sqllab': True,
  'sqlalchemy_uri': 'trino://trino@trino:8080/delta_lake',
  'uuid': '1ed306c2-94d3-4b24-8065-f8ee31be8b8a'}}

In [58]:
database_id = r.json()["id"]

In [ ]:
payload = {
    "database": database_id,
    "schema": "default"
}

for table in ["users", "bronze_reviews_igromania", "bronze_reviews_metacritic", "silver_reviews", "golden_review_facts"]:
    print(f"Registering {table}")
    payload["table_name"] = table
    
    r = requests.post('http://superset:8088/api/v1/dataset', headers=headers, json=payload, cookies=cookies)
    if r.status_code != 201:
        print(r.json())
        assert False

In [ ]:
payload = {
    "database": database_id,
    "schema": "default",
    "table_name": "facts_enriched",
    "is_managed_externally": False,
    "external_url": None,
    "sql": """
SELECT review_id
     , r.app_id
     , f.user_id
     , username
     , gender
     , dob
     , title
     , date_release
     , win
     , mac
     , linux
     , price_final
  FROM golden_review_facts f
  JOIN users u ON (u.id = f.user_id)
  JOIN games g USING (app_id)
  JOIN silver_reviews r ON (r.id = f.review_id)
"""
}
r = requests.post('http://superset:8088/api/v1/dataset', headers=headers, json=payload, cookies=cookies)

assert r.status_code == 201

In [ ]:
# spark.table("golden_review_facts").write.format("delta").mode("overwrite").saveAsTable("my_gold_facts")

In [ ]:
datasource_id = r.json()["id"]
datasource_uid = r.json()["data"]["uid"]

In [ ]:
payload = {
    "params": f"""{{
        "datasource": "{datasource_uid}",
        "viz_type": "echarts_timeseries_bar",
        "x_axis": "date_release",
        "time_grain_sqla": "P1Y",
        "x_axis_sort_asc": true,
        "x_axis_sort_series": "name",
        "x_axis_sort_series_ascending": true,
        "metrics": [
            {{
                "expressionType": "SIMPLE",
                "column": {{
                    "advanced_data_type": null,
                    "certification_details": null,
                    "certified_by": null,
                    "column_name": "review_id",
                    "description": null,
                    "expression": null,
                    "filterable": true,
                    "groupby": true,
                    "id": 20,
                    "is_certified": false,
                    "is_dttm": false,
                    "python_date_format": null,
                    "type": "VARCHAR",
                    "type_generic": 1,
                    "verbose_name": null,
                    "warning_markdown": null
                }},
                "aggregate": "COUNT_DISTINCT",
                "sqlExpression": null,
                "datasourceWarning": false,
                "hasCustomLabel": false,
                "label": "COUNT_DISTINCT(review_id)",
                "optionName": "metric_x2g7cq064tb_vpz0jzxjivl"
            }}
        ],
        "groupby": [
            "gender"
        ],
        "adhoc_filters": [
            {{
                "expressionType": "SIMPLE",
                "subject": "date_release",
                "operator": "TEMPORAL_RANGE",
                "comparator": "No filter",
                "clause": "WHERE",
                "sqlExpression": null,
                "isExtra": false,
                "isNew": false,
                "datasourceWarning": false,
                "filterOptionName": "filter_9glfwpxhc85_loni87wp65"
            }}
        ],
        "order_desc": true,
        "row_limit": 50000,
        "truncate_metric": true,
        "show_empty_columns": true,
        "comparison_type": "values",
        "annotation_layers": [],
        "forecastPeriods": 10,
        "forecastInterval": 0.8,
        "orientation": "vertical",
        "x_axis_title_margin": 15,
        "y_axis_title_margin": 15,
        "y_axis_title_position": "Left",
        "sort_series_type": "sum",
        "color_scheme": "supersetColors",
        "only_total": true,
        "show_legend": true,
        "legendType": "scroll",
        "legendOrientation": "top",
        "x_axis_time_format": "smart_date",
        "y_axis_format": "SMART_NUMBER",
        "y_axis_bounds": [
            null,
            null
        ],
        "rich_tooltip": true,
        "tooltipTimeFormat": "smart_date",
        "extra_form_data": {{}},
        "dashboards": []
    }}
    """,
    "slice_name": "Reviews by Gender",
    "viz_type": "echarts_timeseries_bar",
    "datasource_id": datasource_id,
    "datasource_type": "table",
    "dashboards": [],
    "query_context": """{
        "datasource": {
            "id": 6,
            "type": "table"
        },
        "force": false,
        "queries": [
            {
                "filters": [
                    {
                        "col": "date_release",
                        "op": "TEMPORAL_RANGE",
                        "val": "No filter"
                    }
                ],
                "extras": {
                    "having": "",
                    "where": ""
                },
                "applied_time_extras": {},
                "columns": [
                    {
                        "timeGrain": "P1Y",
                        "columnType": "BASE_AXIS",
                        "sqlExpression": "date_release",
                        "label": "date_release",
                        "expressionType": "SQL"
                    },
                    "gender"
                ],
                "metrics": [
                    {
                        "expressionType": "SIMPLE",
                        "column": {
                            "advanced_data_type": null,
                            "certification_details": null,
                            "certified_by": null,
                            "column_name": "review_id",
                            "description": null,
                            "expression": null,
                            "filterable": true,
                            "groupby": true,
                            "id": 20,
                            "is_certified": false,
                            "is_dttm": false,
                            "python_date_format": null,
                            "type": "VARCHAR",
                            "type_generic": 1,
                            "verbose_name": null,
                            "warning_markdown": null
                        },
                        "aggregate": "COUNT_DISTINCT",
                        "sqlExpression": null,
                        "datasourceWarning": false,
                        "hasCustomLabel": false,
                        "label": "COUNT_DISTINCT(review_id)",
                        "optionName": "metric_x2g7cq064tb_vpz0jzxjivl"
                    }
                ],
                "orderby": [
                    [
                        {
                            "expressionType": "SIMPLE",
                            "column": {
                                "advanced_data_type": null,
                                "certification_details": null,
                                "certified_by": null,
                                "column_name": "review_id",
                                "description": null,
                                "expression": null,
                                "filterable": true,
                                "groupby": true,
                                "id": 20,
                                "is_certified": false,
                                "is_dttm": false,
                                "python_date_format": null,
                                "type": "VARCHAR",
                                "type_generic": 1,
                                "verbose_name": null,
                                "warning_markdown": null
                            },
                            "aggregate": "COUNT_DISTINCT",
                            "sqlExpression": null,
                            "datasourceWarning": false,
                            "hasCustomLabel": false,
                            "label": "COUNT_DISTINCT(review_id)",
                            "optionName": "metric_x2g7cq064tb_vpz0jzxjivl"
                        },
                        false
                    ]
                ],
                "annotation_layers": [],
                "row_limit": 250,
                "series_columns": [
                    "gender"
                ],
                "series_limit": 0,
                "order_desc": true,
                "url_params": {},
                "custom_params": {},
                "custom_form_data": {},
                "time_offsets": [],
                "post_processing": [
                    {
                        "operation": "pivot",
                        "options": {
                            "index": [
                                "date_release"
                            ],
                            "columns": [
                                "gender"
                            ],
                            "aggregates": {
                                "COUNT_DISTINCT(review_id)": {
                                    "operator": "mean"
                                }
                            },
                            "drop_missing_columns": false
                        }
                    },
                    {
                        "operation": "rename",
                        "options": {
                            "columns": {
                                "COUNT_DISTINCT(review_id)": null
                            },
                            "level": 0,
                            "inplace": true
                        }
                    },
                    {
                        "operation": "flatten"
                    }
                ]
            }
        ],
        "form_data": {
            "datasource": "6__table",
            "viz_type": "echarts_timeseries_bar",
            "x_axis": "date_release",
            "time_grain_sqla": "P1Y",
            "x_axis_sort_asc": true,
            "x_axis_sort_series": "name",
            "x_axis_sort_series_ascending": true,
            "metrics": [
                {
                    "expressionType": "SIMPLE",
                    "column": {
                        "advanced_data_type": null,
                        "certification_details": null,
                        "certified_by": null,
                        "column_name": "review_id",
                        "description": null,
                        "expression": null,
                        "filterable": true,
                        "groupby": true,
                        "id": 20,
                        "is_certified": false,
                        "is_dttm": false,
                        "python_date_format": null,
                        "type": "VARCHAR",
                        "type_generic": 1,
                        "verbose_name": null,
                        "warning_markdown": null
                    },
                    "aggregate": "COUNT_DISTINCT",
                    "sqlExpression": null,
                    "datasourceWarning": false,
                    "hasCustomLabel": false,
                    "label": "COUNT_DISTINCT(review_id)",
                    "optionName": "metric_x2g7cq064tb_vpz0jzxjivl"
                }
            ],
            "groupby": [
                "gender"
            ],
            "adhoc_filters": [
                {
                    "expressionType": "SIMPLE",
                    "subject": "date_release",
                    "operator": "TEMPORAL_RANGE",
                    "comparator": "No filter",
                    "clause": "WHERE",
                    "sqlExpression": null,
                    "isExtra": false,
                    "isNew": false,
                    "datasourceWarning": false,
                    "filterOptionName": "filter_9glfwpxhc85_loni87wp65"
                }
            ],
            "order_desc": true,
            "row_limit": 50000,
            "truncate_metric": true,
            "show_empty_columns": true,
            "comparison_type": "values",
            "annotation_layers": [],
            "forecastPeriods": 10,
            "forecastInterval": 0.8,
            "orientation": "vertical",
            "x_axis_title_margin": 15,
            "y_axis_title_margin": 15,
            "y_axis_title_position": "Left",
            "sort_series_type": "sum",
            "color_scheme": "supersetColors",
            "only_total": true,
            "show_legend": true,
            "legendType": "scroll",
            "legendOrientation": "top",
            "x_axis_time_format": "smart_date",
            "y_axis_format": "SMART_NUMBER",
            "y_axis_bounds": [
                null,
                null
            ],
            "rich_tooltip": true,
            "tooltipTimeFormat": "smart_date",
            "extra_form_data": {},
            "dashboards": [],
            "force": false,
            "result_format": "json",
            "result_type": "full"
        },
        "result_format": "json",
        "result_type": "full"
    }
    """
}

In [ ]:
r = requests.post('http://superset:8088/api/v1/chart', headers=headers, json=payload, cookies=cookies)
assert r.status_code == 201

In [ ]:
golden_review_facts_stream.awaitTermination(timeout=5)
golden_review_facts_stream.stop()

In [ ]:
silver_reviews_from_metacritic_stream.awaitTermination(timeout=5)
silver_reviews_from_metacritic_stream.stop()

In [16]:
silver_reviews_from_igromania_stream.awaitTermination(timeout=5)
silver_reviews_from_igromania_stream.stop()

In [ ]:
bronze_reviews_igromania_stream.awaitTermination(timeout=5)
bronze_reviews_igromania_stream.stop()

In [ ]:
bronze_reviews_metacritic_stream.awaitTermination(timeout=5)
bronze_reviews_metacritic_stream.stop()

In [ ]:
# (
# users_df
#     .write
#     .format("delta")
#     .partitionBy("gender")
#     .mode("overwrite")
#     .saveAsTable("users")
# )

In [ ]:
# DeltaTable.forName(spark, "reviews_igromania").optimize().executeZOrderBy("app_id")
# DeltaTable.forName(spark, "reviews_metacritic").optimize().executeZOrderBy("app_id")
# DeltaTable.forPath(spark, "data/steam/games").optimize().executeZOrderBy("app_id")

### Серебрянный слой

In [ ]:
# spark.sql("""
# CREATE OR REPLACE TABLE reviews
# USING DELTA
# PARTITIONED BY (source, rating)
# --ZORDER BY  app_id
# AS (
#   SELECT id
#        , app_id
#        , CAST(from_unixtime(review_date_unix) AS TIMESTAMP) review_date
#        , review
#        , ROUND(rating * 100, 0) rating
#        , user_id
#        , 'IGROMANIA' source
#     FROM reviews_igromania
#    WHERE 1 != 1

#   UNION ALL

#   SELECT id
#        , app_id
#        , to_timestamp(review_date) review_date
#        , review
#        , ROUND(rating, 0) rating
#        , user_id
#        , 'METACRITIC' source
#     FROM reviews_metacritic
#    WHERE 1 != 1
# )
# """)

In [ ]:
spark.sql("DROP TABLE IF EXISTS reviews_silver")

In [ ]:
spark.sql("""
CREATE OR REPLACE TABLE reviews_silver (
    id STRING NOT NULL,
    app_id INT NOT NULL,
    review_ts TIMESTAMP NOT NULL,
    review STRING,
    rating INT,
    user_id INT NOT NULL,
    source STRING NOT NULL
)
USING DELTA
PARTITIONED BY (source, rating)
""")

In [ ]:
spark.sql("DESC EXTENDED reviews_silver").show()

In [ ]:
reviews_silver = DeltaTable.forName(spark, "reviews_silver")

In [ ]:
reviews_bronze_igromania = spark.sql("""
SELECT 'IGROMANIA' || id id
     , app_id
     , CAST(from_unixtime(review_date_unix) AS TIMESTAMP) review_ts
     , review
     , ROUND(rating * 10, 0) rating
     , user_id
     , 'IGROMANIA' source
  FROM reviews_igromania
""")

In [ ]:
reviews_bronze_metacritic = spark.sql("""
SELECT 'METACRITIC' || id id
     , app_id
     , to_timestamp(review_date) review_ts
     , review
     , ROUND(rating, 0) rating
     , user_id
     , 'METACRITIC' source
  FROM reviews_metacritic
""")

In [ ]:
(
reviews_silver.alias("silver")
  .merge(reviews_bronze_igromania.alias("bronze"), col("silver.id") == col("bronze.id"))
  .whenMatchedUpdateAll()
  .whenNotMatchedInsertAll()
  .execute()
)

In [ ]:
(
reviews_silver.alias("silver")
  .merge(reviews_bronze_metacritic.alias("bronze"), col("silver.id") == col("bronze.id"))
  .whenMatchedUpdateAll()
  .whenNotMatchedInsertAll()
  .execute()
)

In [ ]:
spark.table("reviews_silver").count()

In [ ]:
spark.table("reviews_silver").groupBy("id").count().where("count > 1").show()

In [ ]:
# reviews_silver.optimize().executeZOrderBy("app_id")

### Золотой слой - Golden Layer

In [ ]:
spark.sql("DROP TABLE IF EXISTS game_review_facts_golden")

In [ ]:
spark.sql("""
CREATE OR REPLACE TABLE game_review_facts_golden (
    user_id INT NOT NULL,
    app_id INT NOT NULL,
    review_id STRING NOT NULL,
    ts TIMESTAMP NOT NULL
)
USING DELTA
""")

In [ ]:
spark.sql("""
CREATE OR REPLACE TABLE game_review_facts_golden
USING DELTA
AS (
    SELECT user_id
         , app_id
         , id review_id
      FROM reviews_silver
)
""")

In [ ]:
spark.sql("DESCRIBE EXTENDED game_review_facts_golden").show(100, False)

In [ ]:
golden_facts_stream = (
    spark.readStream
        .format("delta")
        # .table("reviews_silver")
        .load("/user/jovyan/data/spark-warehouse/reviews_silver")    
        .writeStream
        .format("console")
)
golden_facts_stream.start()

In [ ]:
spark.range(10).show()

In [ ]:
(
game_review_facts_golden.alias("gold")
  .merge(reviews_silver.alias("silver"), col("gold.user_id") == col("bronze.id"))
  .whenMatchedUpdateAll()
  .whenNotMatchedInsertAll()
  .execute()
)

In [ ]:
reviews_bronze_igromania = spark.sql("""
SELECT 'IGROMANIA' || id id
     , app_id
     , CAST(from_unixtime(review_date_unix) AS TIMESTAMP) review_ts
     , review
     , ROUND(rating * 10, 0) rating
     , user_id
     , 'IGROMANIA' source
  FROM reviews_igromania
""")

In [ ]:
reviews_bronze_metacritic = spark.sql("""
SELECT 'METACRITIC' || id id
     , app_id
     , to_timestamp(review_date) review_ts
     , review
     , ROUND(rating, 0) rating
     , user_id
     , 'METACRITIC' source
  FROM reviews_metacritic
""")

In [ ]:
(
reviews_silver.alias("silver")
  .merge(reviews_bronze_igromania.alias("bronze"), col("silver.id") == col("bronze.id"))
  .whenMatchedUpdateAll()
  .whenNotMatchedInsertAll()
  .execute()
)

In [ ]:
(
reviews_silver.alias("silver")
  .merge(reviews_bronze_metacritic.alias("bronze"), col("silver.id") == col("bronze.id"))
  .whenMatchedUpdateAll()
  .whenNotMatchedInsertAll()
  .execute()
)

In [ ]:
taxi_df = spark.read.parquet("/user/jovyan/data/taxi")

In [ ]:
taxi_df = spark.read.parquet("/user/jovyan/data/taxi")
taxi = bucketing(taxi_df, "taxi", "passenger_count")

In [ ]:
spark.sql("""
CREATE TABLE IF NOT EXISTS taxi
USING DELTA
PARTITIONED BY (passenger_count)
""").collect()

In [ ]:
spark.catalog.listDatabases()

In [ ]:
spark.sql("DROP TABLE IF EXISTS taxi").collect()